In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
# get all file in the directory

def get_all_files(directory):
    all_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv') and not file.startswith('.') and not file.startswith('cleaned'):
                all_files.append(os.path.join(root, file))
    return all_files

# get all files in current directory

all_files = get_all_files('.')
print(all_files)

# print(all_files[0][-6:-4])

['./session00_202309.csv', './session00_202310.csv', './session00_202311.csv', './session00_202312.csv', './session00_202401.csv', './session00_202402.csv']


In [1]:
# clean redundant columns


for file in all_files:
    df = pd.read_csv(file)
    # get month in file name(last two digits in file name)
    cur_month = file[-6:-4]
    # get rid of rows with empty value in the "Behavior" column
    df = df[df['Behavior'] != '']
    
    # get rid of "Language" column
    df = df.drop(columns=['Language'])
    
    # get rid of "CountryAliasCode" column
    df = df.drop(columns=['CountryAliasCode'])
    
    # get rid of "Version" column
    df = df.drop(columns=['Version'])
    
    # get rid of "UTMSource" column
    df = df.drop(columns=['UTMSource'])
    
    # get rid of "UTMMedium" column
    df = df.drop(columns=['UTMMedium'])
    
    # get rid of "UTMName" column
    df = df.drop(columns=['UTMName'])

    #get rid of "CurrencyCode" column
    df = df.drop(columns=['CurrencyCode'])


    # get rid of "RegisterTunnel" rows is not ''
    df = df[df['RegisterTunnel'] != '']

    #get rid of non-numeric value in "HitTime" column    
    df = df[pd.to_numeric(df['HitTime'], errors='coerce').notnull()]

    # turn "HitTime" clumn into datetime(now is Unix Timestamp)
    df['HitTime'] = pd.to_datetime(df['HitTime'], unit='ms')
    
    # df['HitTime'] add 8 hours
    df['HitTime'] = df['HitTime'] + pd.Timedelta(hours=8)

    # same with "EventTime" column
    df['EventTime'] = pd.to_datetime(df['EventTime'], unit='ms')
    df['EventTime'] = df['EventTime'] + pd.Timedelta(hours=8)

    # # check if "HitTime" and "EventTime" are in the same month as cur_month
    # df = df[df['HitTime'].dt.month == int(cur_month)]
    # df = df[df['EventTime'].dt.month == int(cur_month)]
    
    #get rid of  the beginning "./"  in string "file"
    file = file[2:]

    # # check if the "behavior" folder exists
    # if not os.path.exists('behavior'):
    #     os.makedirs('behavior')
    
    # # write to a new csv file, in a "behavior" folder
    # df.to_csv('behavior/cleaned_' + file, index=False)

    unique_year_month = df['HitTime'].dt.strftime('%Y-%m').unique()

    # get year-month from unique_year_month, sort it and get the smaller value as current_year_month
    if len(unique_year_month) != 0:
        current_year_month = sorted(unique_year_month)[0]

    
    for year_month in unique_year_month:
        # get the data in the same year_month, use tmp_df to avoid the data loss
        tmp_df = df[df['HitTime'].dt.strftime('%Y-%m') == year_month]

        # 10000 rows per chunk, write to a new csv file, under behavior/year_month folder
        # check if the "behavior" folder exists
        if not os.path.exists('behavior/' + year_month):
            os.makedirs('behavior/' + year_month)
        # check if the "year" folder exists
        if not os.path.exists('behavior/' + year_month):
            os.makedirs('behavior/' + year_month)
        chunk_size = 100000
        chunks = [tmp_df[i:i+chunk_size] for i in range(0, tmp_df.shape[0], chunk_size)]
        for i, chunk in enumerate(chunks):
            chunk.to_csv('behavior/' + year_month + '/cleaned_' + year_month + '_' + str(i) + '_from_' + current_year_month + '.csv', index=False)



In [10]:
# # clean redundant columns

# all_files = ['session00_202402.csv']


# for file in all_files:
#     df = pd.read_csv(file)
#     # get month in file name(last two digits in file name)
#     cur_month = file[-6:-4]
#     # get rid of rows with empty value in the "Behavior" column
#     df = df[df['Behavior'] != '']
    
#     # get rid of "Language" column
#     df = df.drop(columns=['Language'])
    
#     # get rid of "CountryAliasCode" column
#     df = df.drop(columns=['CountryAliasCode'])
    
#     # get rid of "Version" column
#     df = df.drop(columns=['Version'])
    
#     # get rid of "UTMSource" column
#     df = df.drop(columns=['UTMSource'])
    
#     # get rid of "UTMMedium" column
#     df = df.drop(columns=['UTMMedium'])
    
#     # get rid of "UTMName" column
#     df = df.drop(columns=['UTMName'])

#     #get rid of "CurrencyCode" column
#     df = df.drop(columns=['CurrencyCode'])


#     # get rid of "RegisterTunnel" rows is not ''
#     df = df[df['RegisterTunnel'] != '']

#     #get rid of non-numeric value in "HitTime" column    
#     df = df[pd.to_numeric(df['HitTime'], errors='coerce').notnull()]

#     # turn "HitTime" clumn into datetime(now is Unix Timestamp)
#     df['HitTime'] = pd.to_datetime(df['HitTime'], unit='ms')
    
#     # df['HitTime'] add 8 hours
#     df['HitTime'] = df['HitTime'] + pd.Timedelta(hours=8)

#     # same with "EventTime" column
#     df['EventTime'] = pd.to_datetime(df['EventTime'], unit='ms')
#     df['EventTime'] = df['EventTime'] + pd.Timedelta(hours=8)

#     # # check if "HitTime" and "EventTime" are in the same month as cur_month
#     # df = df[df['HitTime'].dt.month == int(cur_month)]
#     # df = df[df['EventTime'].dt.month == int(cur_month)]
    
#     #get rid of  the beginning "./"  in string "file"
#     file = file[2:]

#     # # check if the "behavior" folder exists
#     # if not os.path.exists('behavior'):
#     #     os.makedirs('behavior')
    
#     # # write to a new csv file, in a "behavior" folder
#     # df.to_csv('behavior/cleaned_' + file, index=False)

#     unique_year_month = df['HitTime'].dt.strftime('%Y-%m').unique()

#     # get year-month from unique_year_month, sort it and get the smaller value as current_year_month
#     if len(unique_year_month) != 0:
#         current_year_month = sorted(unique_year_month)[0]

    
#     for year_month in unique_year_month:
#         # get the data in the same year_month, use tmp_df to avoid the data loss
#         tmp_df = df[df['HitTime'].dt.strftime('%Y-%m') == year_month]

#         # 10000 rows per chunk, write to a new csv file, under behavior/year_month folder
#         # check if the "behavior" folder exists
#         if not os.path.exists('behavior/' + year_month):
#             os.makedirs('behavior/' + year_month)
#         # check if the "year" folder exists
#         if not os.path.exists('behavior/' + year_month):
#             os.makedirs('behavior/' + year_month)
#         chunk_size = 100000
#         chunks = [tmp_df[i:i+chunk_size] for i in range(0, tmp_df.shape[0], chunk_size)]
#         for i, chunk in enumerate(chunks):
#             chunk.to_csv('behavior/' + year_month + '/cleaned_' + year_month + '_' + str(i) + '_from_' + current_year_month + '.csv', index=False)



/var/folders/7g/rxqxvrmd0z1bnnny3trbtfz80000gn/T/ipykernel_6155/3150213844.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


['2024-02']
[<class 'str'>]
